In [1]:
import cirq
import numpy as np
from sklearn.preprocessing import normalize
import sympy as sp
from cirq.contrib.svg import SVGCircuit
import tensorflow_quantum as tfq
from tensorflow_quantum.python import util
import re
import collections

2022-07-14 17:40:59.171869: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-14 17:40:59.171908: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-07-14 17:41:01.566723: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-07-14 17:41:01.566785: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (bhagvada): /proc/driver/nvidia/version does not exist
2022-07-14 17:41:01.567217: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
T

In [2]:
def beta(s, j, x):
    index_num = (2*j-1)*(2**(s-1))
    index_den = (j-1)*(2**s)
    
    num = np.sqrt(np.sum(abs(x[index_num : index_num+2**(s-1)])**2))
    den = np.sqrt(np.sum(abs(x[index_den : index_den+2**(s)])**2))
    
    if den == 0:
        beta = 0
    else:
        beta = 2*np.arcsin(num/den)
    print(beta)
    return beta

In [3]:
def locate_x(curr_j, prev_j, length):
    curr_bin = bin(curr_j)[2:].zfill(length)
    prev_bin = bin(prev_j)[2:].zfill(length)
    return [i for i, (x,y) in enumerate(zip(curr_bin,prev_bin)) if x!=y]

In [4]:
def amplitude_embedding(x):
    n = int(np.log2(len(x)))
    qubits = cirq.GridQubit.rect(1, n)
    circuit = cirq.Circuit()
    
    circuit += cirq.ry(beta(n, 1, x))(qubits[0])
    
    for i in range(1,n):
        # We can have at most i control bits
        # Total possibilities is therefore 2^i
        controls = 2**i
        
        control_qubits = [qubits[c] for c in range(i+1)]
        circuit += cirq.ControlledGate(sub_gate=cirq.ry(beta(n-i, controls, x)), 
                                       num_controls=len(control_qubits)-1)(*control_qubits)
        
        for j in range(1, controls):
            for loc in locate_x(controls-j-1, controls-j, i):
                circuit += cirq.X(qubits[loc])
                
            circuit += cirq.ControlledGate(sub_gate=cirq.ry(beta(n-i, controls-j, x)), 
                                       num_controls=len(control_qubits)-1)(*control_qubits)
            
        for k in range(i):
            circuit += cirq.X(qubits[k])

    return circuit

In [5]:
x = np.arange(1,5,1)
d = np.sqrt(np.sum(np.square(x)))
x = x/d
print('amplitudes:',x)
print('probs:',np.square(x))
qubits = cirq.GridQubit.rect(1,int(np.ceil(np.log2(len(x)))))
qc = cirq.Circuit()
qc += amplitude_embedding(x)
qc += cirq.measure(*qubits, key='result')

amplitudes: [0.18257419 0.36514837 0.54772256 0.73029674]
probs: [0.03333333 0.13333333 0.3        0.53333333]
2.300523983021863
1.8545904360032246
2.2142974355881813


In [6]:
qc

(0, 0): ───Ry(0.732π)───@───────────X───@────────────X───M('result')───
                        │               │                │
(0, 1): ────────────────Ry(0.59π)───────Ry(0.705π)───────M─────────────

In [7]:
s=cirq.Simulator()
shots = 10000
samples=s.run(qc, repetitions=shots)
res = dict(samples.histogram(key="result"))
for key, value in res.items():
    res[key] = value/shots
res

{2: 0.2917, 1: 0.1369, 3: 0.5369, 0: 0.0345}

In [8]:
od = collections.OrderedDict(sorted(res.items()))

for k, v in od.items():
    print(k,v)

0 0.0345
1 0.1369
2 0.2917
3 0.5369


In [9]:
x = np.arange(5,9,1)
d = np.sqrt(np.sum(np.square(x)))
x = x/d
print('amplitudes:',x)
print('probs:',np.square(x))
qubits = cirq.GridQubit.rect(1,int(np.ceil(np.log2(len(x)))))
qc = cirq.Circuit()
qc += amplitude_embedding(x)
qc += cirq.measure(*qubits, key='result')

amplitudes: [0.37904902 0.45485883 0.53066863 0.60647843]
probs: [0.14367816 0.20689655 0.2816092  0.36781609]
1.8742842836248206
1.7039326543465443
1.7521161011963868


In [10]:
qc

(0, 0): ───Ry(0.597π)───@────────────X───@────────────X───M('result')───
                        │                │                │
(0, 1): ────────────────Ry(0.542π)───────Ry(0.558π)───────M─────────────

In [11]:
s=cirq.Simulator()
shots = 10000
samples=s.run(qc, repetitions=shots)
res = dict(samples.histogram(key="result"))
for key, value in res.items():
    res[key] = value/shots
res

{2: 0.2751, 1: 0.2069, 3: 0.3751, 0: 0.1429}

## Amplitude Encoding using module

In [12]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

from importlib.util import find_spec
if find_spec("qml_hep_lhc") is None:
    import sys
    sys.path.append('..')

In [13]:
from qml_hep_lhc.encodings import AmplitudeMap
from qml_hep_lhc.data import ElectronPhoton
import sympy as sp
import argparse
from qml_hep_lhc.layers.utils import get_count_of_qubits, get_num_in_symbols
from qml_hep_lhc.layers.utils import symbols_in_expr_map, resolve_formulas
from tensorflow import pad, constant
import tensorflow as tf

In [14]:
n_qubits = get_count_of_qubits('AmplitudeMap',16)
n_inputs = get_num_in_symbols('AmplitudeMap', 16)

n_qubits, n_inputs

(4, 16)

In [15]:
in_symbols = sp.symbols(f'x0:{n_inputs}')
in_symbols = np.asarray(in_symbols).reshape(1,n_inputs)

in_symbols

array([[x0, x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12, x13, x14,
        x15]], dtype=object)

In [16]:
qubits = cirq.GridQubit.rect(1,n_qubits)
circuit = cirq.Circuit()
circuit += AmplitudeMap().build(qubits,in_symbols[0])

In [17]:
circuit

(0, 0): ───Ry(2*asin((Abs(x10)**2 + Abs(x11)**2 + Abs(x12)**2 + Abs(x13)**2 + Abs(x14)**2 + Abs(x15)**2 + Abs(x8)**2 + Abs(x9)**2)**0.5*(Abs(x0)**2 + Abs(x1)**2 + Abs(x10)**2 + Abs(x11)**2 + Abs(x12)**2 + Abs(x13)**2 + Abs(x14)**2 + Abs(x15)**2 + Abs(x2)**2 + Abs(x3)**2 + Abs(x4)**2 + Abs(x5)**2 + Abs(x6)**2 + Abs(x7)**2 + Abs(x8)**2 + Abs(x9)**2)**(-0.5)))───@────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────X───@──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────X───@──────────────────────────────────────────────────────────────────────────────────────────────────────────────────@────────────────────────────────────────────────────────────────────────────────────────────────────────────X───@────────────────────────────────────────────────────────────────────────────────────────────────────────────@────────────────────────────────────────────────────────────────────────────────────────────────────────X───@───────────────────────────────────────────────────────────────────@───────────────────────────────────────────────────────────────────@───────────────────────────────────────────────────────────────────@────────────────────────────────────────────────────────────X───@────────────────────────────────────────────────────────────────@────────────────────────────────────────────────────────────────@────────────────────────────────────────────────────────────────@────────────────────────────────────────────────────────────X───
                                                                                                                                                                                                                                                                                                                                                                          │                                                                                                                                                                                                    │                                                                                                                                                                                          │                                                                                                                  │                                                                                                                │                                                                                                            │                                                                                                            │                                                                   │                                                                   │                                                                   │                                                                │                                                                │                                                                │                                                                │
(0, 1): ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────Ry(2*asin((Abs(x12)**2 + Abs(x13)**2 + Abs(x14)**2 + Abs(x15)**2)**0.5*(Abs(x10)**2 + Abs(x11)**2 + Abs(x12)**2 + Abs(x13)**2 + Abs(x14)**2 + Abs(x15)**2 + Abs(x8)**2 + Abs(x9)**2)**(-0.5)))───────Ry(2*asin((Abs(x4)**2 + Abs(x5)**2 + Abs(x6)**2 + Abs(x7)**2)

In [18]:
circuit, expr_map = cirq.flatten(circuit)
raw_in_symbols = symbols_in_expr_map(expr_map)
data_expr = list(expr_map)

In [19]:
raw_in_symbols

[x0, x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12, x13, x14, x15]

In [20]:
data_expr

[2*asin((Abs(x10)**2 + Abs(x11)**2 + Abs(x12)**2 + Abs(x13)**2 + Abs(x14)**2 + Abs(x15)**2 + Abs(x8)**2 + Abs(x9)**2)**0.5*(Abs(x0)**2 + Abs(x1)**2 + Abs(x10)**2 + Abs(x11)**2 + Abs(x12)**2 + Abs(x13)**2 + Abs(x14)**2 + Abs(x15)**2 + Abs(x2)**2 + Abs(x3)**2 + Abs(x4)**2 + Abs(x5)**2 + Abs(x6)**2 + Abs(x7)**2 + Abs(x8)**2 + Abs(x9)**2)**(-0.5))/pi,
 2*asin((Abs(x12)**2 + Abs(x13)**2 + Abs(x14)**2 + Abs(x15)**2)**0.5*(Abs(x10)**2 + Abs(x11)**2 + Abs(x12)**2 + Abs(x13)**2 + Abs(x14)**2 + Abs(x15)**2 + Abs(x8)**2 + Abs(x9)**2)**(-0.5))/pi,
 2*asin((Abs(x4)**2 + Abs(x5)**2 + Abs(x6)**2 + Abs(x7)**2)**0.5*(Abs(x0)**2 + Abs(x1)**2 + Abs(x2)**2 + Abs(x3)**2 + Abs(x4)**2 + Abs(x5)**2 + Abs(x6)**2 + Abs(x7)**2)**(-0.5))/pi,
 2*asin((Abs(x14)**2 + Abs(x15)**2)**0.5*(Abs(x12)**2 + Abs(x13)**2 + Abs(x14)**2 + Abs(x15)**2)**(-0.5))/pi,
 2*asin((Abs(x10)**2 + Abs(x11)**2)**0.5*(Abs(x10)**2 + Abs(x11)**2 + Abs(x8)**2 + Abs(x9)**2)**(-0.5))/pi,
 2*asin((Abs(x6)**2 + Abs(x7)**2)**0.5*(Abs(x4)**2 + Abs(x

In [21]:
circuit

(0, 0): ───Ry(pi*<2*asin((Abs(x10)**2 + Abs(x11)**2 + Abs(x12)**2 + Abs(x13)**2 + Abs(x14)**2 + Abs(x15)**2 + Abs(x8)**2 + Abs(x9)**2)**0.5*(Abs(x0)**2 + Abs(x1)**2 + Abs(x10)**2 + Abs(x11)**2 + Abs(x12)**2 + Abs(x13)**2 + Abs(x14)**2 + Abs(x15)**2 + Abs(x2)**2 + Abs(x3)**2 + Abs(x4)**2 + Abs(x5)**2 + Abs(x6)**2 + Abs(x7)**2 + Abs(x8)**2 + Abs(x9)**2)**(-0.5))/pi>)───@────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────X───@──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────X───@──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────@────────────────────────────────────────────────────────────────────────────────────────────────────────────────────X───@────────────────────────────────────────────────────────────────────────────────────────────────────────────────────@────────────────────────────────────────────────────────────────────────────────────────────────────────────────X───@───────────────────────────────────────────────────────────────────────────@───────────────────────────────────────────────────────────────────────────@───────────────────────────────────────────────────────────────────────────@────────────────────────────────────────────────────────────────────X───@────────────────────────────────────────────────────────────────────────@────────────────────────────────────────────────────────────────────────@────────────────────────────────────────────────────────────────────────@────────────────────────────────────────────────────────────────────X───
                                                                                                                                                                                                                                                                                                                                                                                  │                                                                                                                                                                                                            │                                                                                                                                                                                                  │                                                                                                                          │                                                                                                                        │                                                                                                                    │                                                                                                                    │                                                                           │                                                                           │                                                                           │                                                                        │                                                                        │                                                                        │                                                                        │
(0, 1): ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────Ry(pi*<2*asin((Abs

In [22]:
list(expr_map.values())

[<2*asin((Abs(x10)**2 + Abs(x11)**2 + Abs(x12)**2 + Abs(x13)**2 + Abs(x14)**2 + Abs(x15)**2 + Abs(x8)**2 + Abs(x9)**2)**0.5*(Abs(x0)**2 + Abs(x1)**2 + Abs(x10)**2 + Abs(x11)**2 + Abs(x12)**2 + Abs(x13)**2 + Abs(x14)**2 + Abs(x15)**2 + Abs(x2)**2 + Abs(x3)**2 + Abs(x4)**2 + Abs(x5)**2 + Abs(x6)**2 + Abs(x7)**2 + Abs(x8)**2 + Abs(x9)**2)**(-0.5))/pi>,
 <2*asin((Abs(x12)**2 + Abs(x13)**2 + Abs(x14)**2 + Abs(x15)**2)**0.5*(Abs(x10)**2 + Abs(x11)**2 + Abs(x12)**2 + Abs(x13)**2 + Abs(x14)**2 + Abs(x15)**2 + Abs(x8)**2 + Abs(x9)**2)**(-0.5))/pi>,
 <2*asin((Abs(x4)**2 + Abs(x5)**2 + Abs(x6)**2 + Abs(x7)**2)**0.5*(Abs(x0)**2 + Abs(x1)**2 + Abs(x2)**2 + Abs(x3)**2 + Abs(x4)**2 + Abs(x5)**2 + Abs(x6)**2 + Abs(x7)**2)**(-0.5))/pi>,
 <2*asin((Abs(x14)**2 + Abs(x15)**2)**0.5*(Abs(x12)**2 + Abs(x13)**2 + Abs(x14)**2 + Abs(x15)**2)**(-0.5))/pi>,
 <2*asin((Abs(x10)**2 + Abs(x11)**2)**0.5*(Abs(x10)**2 + Abs(x11)**2 + Abs(x8)**2 + Abs(x9)**2)**(-0.5))/pi>,
 <2*asin((Abs(x6)**2 + Abs(x7)**2)**0.5*(Abs(x4)

In [23]:
input_resolver = resolve_formulas(data_expr, raw_in_symbols)

In [46]:
x = np.array([0.  ,       0.  ,       0.  ,       0. ,        0.    ,     0.,
  0.8688998,  0.   ,      0.    ,     0.    ,     0.    ,     0.49498802,
  0.      ,   0.    ,     0.   ,      0.        ], dtype=np.float32)
x = x.reshape(1,-1)
print(x)
print(tf.rank(x))
print(np.sqrt(np.sum(x**2)))
padding = n_inputs - x.shape[1]
if padding:
    print("padded")
    x = pad(x, constant([ [0,0], [0, padding]]), constant_values=1.0)
x, _ = tf.linalg.normalize(x, axis = 1)

[[0.         0.         0.         0.         0.         0.
  0.8688998  0.         0.         0.         0.         0.49498802
  0.         0.         0.         0.        ]]
tf.Tensor(2, shape=(), dtype=int32)
1.0


In [47]:
np.sqrt(np.sum(x**2))

1.0

In [48]:
x.numpy()

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.8688998 , 0.        , 0.        , 0.        ,
        0.        , 0.49498802, 0.        , 0.        , 0.        ,
        0.        ]], dtype=float32)

In [49]:
resolved_x = input_resolver(x)

In [50]:
resolved_x.numpy()

array([[0.3296551, 0.       , 1.       , 0.       , 0.9997801, 1.       ,
        0.       , 0.       , 0.       , 0.9997801, 0.       , 0.       ,
        0.       , 0.       , 0.       ]], dtype=float32)

In [29]:
new_x = [[]]
for i in resolved_x[0]:
    if np.isnan(i.numpy()):
        new_x[0].append(0)
    else:
        new_x[0].append(i.numpy())

In [30]:
new_x

[[0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.49521348,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0]]

In [31]:
resolved_x.shape

TensorShape([1, 15])

In [32]:
resolver = cirq.ParamResolver()

In [33]:
qc = cirq.resolve_parameters(circuit, { j: new_x[0][i]  for i,j  in enumerate(list(expr_map.values()))})
qc

(0, 0): ───Ry(0)───@───────X───@───────X───@───────────@───────X───@────────────────@───────X───@───────────@───────────@───────────@───────X───@───────────@───────────@───────────@───────X───
                   │           │           │           │           │                │           │           │           │           │           │           │           │           │
(0, 1): ───────────Ry(0)───────Ry(π)───────@───────X───@───────X───@────────────X───@───────X───@───────────@───────X───@───────────@───────X───@───────────@───────X───@───────────@───────X───
                                           │           │           │                │           │           │           │           │           │           │           │           │
(0, 2): ───────────────────────────────────Ry(0)───────Ry(0)───────Ry(0.495π)───────Ry(0)───────@───────X───@───────X───@───────X───@───────X───@───────X───@───────X───@───────X───@───────X───
                                                                                                │           │           │           │           │           │           │           │
(0, 3): ────────────────────────────────────────────────────────────────────────────────────────Ry(0)───────Ry(0)───────Ry(0)───────Ry(0)───────Ry(0)───────Ry(π)───────Ry(0)───────Ry(0)───────

In [34]:
qc += cirq.measure(*qubits, key='result')

In [35]:
s=cirq.Simulator()
shots = 10000
samples=s.run(qc, repetitions=shots)
res = dict(samples.histogram(key="result"))
for key, value in res.items():
    res[key] = value/shots
res

{5: 0.5057, 6: 0.4943}

In [36]:
od = collections.OrderedDict(sorted(res.items()))

for k, v in od.items():
    print(k,v)

5 0.5057
6 0.4943


In [37]:
probs = np.square(x)[0]
for i, j in enumerate(probs):
    print(i,j)

0 0.0
1 0.0
2 0.0
3 0.0
4 0.0
5 0.50751835
6 0.49248165
7 0.0
8 0.0
9 0.0
10 0.0
11 0.0
12 0.0
13 0.0
14 0.0
15 0.0


In [38]:
x = tf.constant([5.0, np.nan, 6.8, np.nan, np.inf])
tf.math.is_nan(x)

<tf.Tensor: shape=(5,), dtype=bool, numpy=array([False,  True, False,  True, False])>

In [39]:
tf.constant(0.0, dtype=tf.float32)

<tf.Tensor: shape=(), dtype=float32, numpy=0.0>

In [40]:
a = 0.0
if tf.math.reduce_sum(a):
    print('xsd')
else:
    print('sds')

sds


In [41]:
tf.math.reduce_sum(a)

<tf.Tensor: shape=(), dtype=float32, numpy=0.0>

In [42]:
 0.0 * tf.pow(0.0, 0.0)

<tf.Tensor: shape=(), dtype=float32, numpy=0.0>

In [43]:
x = np.array([
    [1,2,3,4],
    [5,6,7,8]
],dtype=np.float32)

In [44]:
y,_ = tf.linalg.normalize(x, axis = 1)

In [45]:
np.sum(y.numpy()**2)

1.9999999